# Ask the City Use Case: Birth Registration

This notebook demonstrates the AskTheCity Use Case on how to interact with Orchestration Service, enabling the creation of AI-driven workflows by seamlessly integrating various modules, such as templating, large language models (LLMs), data masking and content filtering. We will be elaborating on the following:

1. Initialising the Orchestration Service
2. Data Masking
3. Content Filtering
4. Grounding (Workaround via direct API request)

## 1. Initializing the Orchestration Service

⚠️ Before using the SDK, you need to set up a virtual deployment of the Orchestration Service. Once deployed, you'll have access to a unique endpoint URL (deploymentUrl).

In [1]:
YOUR_API_URL = "https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/db479e9a3c5e8722"

### Step 1: Define the Template and Default Input Values

The Template class is used to define structured message templates for generating dynamic interactions with language models.

In [2]:
sysText='''
You are a City Council Information Officer.

Your Responsibilities:

1. Provide accurate and up-to-date information on city council policies, procedures, and services.
2. Answer citizens' questions and respond to their inquiries in a helpful and courteous manner.
3. Guide citizens through city council processes, such as applying for permits, licenses, or services.
4. Offer suggestions and recommendations to citizens on how to resolve issues or address concerns.
5. Escalate complex or sensitive issues to human customer support agents or city council staff as needed.
6. Continuously learn and improve knowledge base to ensure accuracy and relevance of information provided.

Your Personality:

1. Friendly and approachable tone
2. Empathetic and understanding of citizens' concerns
3. Clear and concise communication style
4. Neutral and impartial in providing information
5. Patient and willing to explain complex information in simple terms

Your Authority:

1. Authorized to provide information on city council policies, procedures, and services.
2. Authorized to guide citizens through city council processes.
3. Not authorized to make decisions or take actions on behalf of the city council.
4. Not authorized to provide personal opinions or biased information.
'''

userText='''
Answer the following question: {{?citizenQuestion}}.
Below the answer, restate the original text as posted by the user.
'''

question='''
Hi, I'm Emma Wilson, and I'd like to register my baby's birth. My details are as follows: I was born Emma Louise Johnson on February 12, 1992, in Sagenai City, and I work as a Marketing Manager in MM Corporation Limited. I married Michael David Wilson on August 15, 2015.
My partner, Michael, was born on June 25, 1990, in London, and he works as a Software Engineer, in SE Technologies.
Our baby, Olivia Rose Wilson, was born on October 10, 2024, at Sagenai City Hospital.
You can reach me at emmawilson@email.com or 07891234567, and my address is 123 Main Street, Sagenai, M1 1AB. Could you please let me know what documents are required for registration?
'''

In [3]:
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue

template = Template(
    messages=[
        SystemMessage(sysText),
        UserMessage(userText),
    ],
    defaults=[
        TemplateValue(name="citizenQuestion", value=question),
    ],
)

### Step 2: Define the LLM

Now that you have YOUR_API_URL, let's walk through a basic orchestration pipeline for a translation task.
The LLM class is used to configure and initialize a language model for generating text based on specific parameters.

ℹ️Note that virtual deployment of the language model is managed automatically by the Orchestration Service, so no additional deployment setup is required on your part.

In [4]:
from gen_ai_hub.orchestration.models.llm import LLM

llm = LLM(name="gpt-4o", version="latest", parameters={"max_tokens": 256, "temperature": 0.2})

This configuration initializes the language model to use the gpt-4o variant with the latest updates. The model will generate responses up to 256 tokens in length and produce more predictable and focused output due to the low temperature setting.

### Step 3: Create the Orchestration Configuration

The OrchestrationConfig class is used to create a configuration that integrates various components, such as templates and language models, into a unified orchestration setup. This configuration specifies how these components work together to achieve the desired workflow.

In [5]:
from gen_ai_hub.orchestration.models.config import OrchestrationConfig

config = OrchestrationConfig(
    template=template,
    llm=llm,
)

### Step 4: Run the Orchestration Request

The OrchestrationService class is used to interact with the orchestration service by providing a configuration and invoking its operations. This service handles the execution of workflows defined by the provided configuration and processes inputs accordingly.

In [6]:
from gen_ai_hub.proxy import get_proxy_client

In [7]:
from gen_ai_hub.orchestration.service import OrchestrationService

orchestration_service = OrchestrationService(api_url=YOUR_API_URL, config=config)

Call the run method with the required template_values. The service will process the input according to the configuration and return the result.

In [8]:
result = orchestration_service.run(template_values=[
    TemplateValue(name="citizenQuestion", value=question)
])
print(result.orchestration_result.choices[0].message.content)

Hi Emma,

Congratulations on the birth of your baby, Olivia Rose Wilson! To register your baby's birth, you will need to provide the following documents:

1. **Proof of Identity for Both Parents**: This can be a passport, driving license, or any other government-issued ID.
2. **Proof of Address**: A recent utility bill, bank statement, or any official document showing your current address.
3. **Marriage Certificate**: Since you mentioned you are married, please bring your marriage certificate.
4. **Birth Certificate of the Baby**: The hospital will typically provide a notification of birth, which you will need to bring.
5. **Parent's Birth Certificates**: It may be helpful to bring your own birth certificates, though this is not always required.

You can register the birth at the local registry office in Sagenai City. It is advisable to call ahead and make an appointment to ensure a smooth process. If you have any further questions or need assistance, please feel free to reach out.

Be

## 2. Data Masking

The Data Masking Module anonymizes or pseudonymizes personally identifiable information (PII) before it is processed by the LLM module. When data is anonymized, all identifying information is replaced with placeholders (e.g., MASKED_ENTITY), and the original data cannot be recovered, ensuring that no trace of the original information is retained. In contrast, pseudonymized data is substituted with unique placeholders (e.g., MASKED_ENTITY_ID), allowing the original information to be restored if needed. In both cases, the masking module identifies sensitive data and replaces it with appropriate placeholders before further processing.

In [9]:
from gen_ai_hub.orchestration.utils import load_text_file
from gen_ai_hub.orchestration.models.data_masking import DataMasking
from gen_ai_hub.orchestration.models.sap_data_privacy_integration import SAPDataPrivacyIntegration, MaskingMethod, \
    ProfileEntity

data_masking = DataMasking(
    providers=[
        SAPDataPrivacyIntegration(
            method=MaskingMethod.ANONYMIZATION,  # or MaskingMethod.PSEUDONYMIZATION
            entities=[
                ProfileEntity.EMAIL,
                ProfileEntity.PHONE,
                ProfileEntity.PERSON,
                ProfileEntity.ORG,
                ProfileEntity.LOCATION,
                ProfileEntity.CREDIT_CARD_NUMBER,
                ProfileEntity.SENSITIVE_DATA
            ]
        )
    ]
)

config = OrchestrationConfig(
    template=Template(
        messages=[
            SystemMessage(sysText),
            UserMessage(userText),
        ]
    ),
    llm=LLM(
        name="gpt-35-turbo",
    ),
    data_masking=data_masking
)

result = orchestration_service.run(
    config=config,
    template_values=[
        TemplateValue(name="citizenQuestion", value=question)
    ]
)

In [10]:
print(result.orchestration_result.choices[0].message.content)

Sure, to register your baby's birth, you will need to provide the following documents:
- Your and your partner's original birth certificates 
- Your marriage certificate or civil partnership certificate (if applicable)
- Your and your partner's passports
- Proof of your address, such as a utility bill or bank statement

Original text:
Hi, I'm MASKED_PERSON, and I'd like to register my baby's birth. My details are as follows: MASKED_SENSITIVE_DATA was born MASKED_PERSON MASKED_PERSON on February 12, 1992, in MASKED_LOCATION, and MASKED_SENSITIVE_DATA work as a Marketing Manager in MASKED_ORG. MASKED_SENSITIVE_DATA married MASKED_PERSON on August 15, 2015.
My partner, MASKED_PERSON, was born on June 25, 1990, in MASKED_LOCATION, and MASKED_SENSITIVE_DATA works as a Software Engineer, in MASKED_ORG.
Our baby, MASKED_PERSON, was born on October 10, 2024, at MASKED_LOCATION Hospital.
MASKED_SENSITIVE_DATA can reach MASKED_SENSITIVE_DATA at MASKED_EMAIL or 07891234567, and my address is 123 

## 3. Content Filtering

The Content Filtering Module can be configured to filter both the input to the LLM module (input filter) and the output generated by the LLM (output filter). The module uses predefined classification services to detect inappropriate or unwanted content, allowing flexible configuration through customizable thresholds. These thresholds can be set to control the sensitivity of filtering, ensuring that content meets desired standards before it is processed or returned as output.

In [11]:
hatefulText='''
LISTEN UP, YOU INCOMPETENT BUREAUCRAT! I'M EMMA WILSON, AND I DEMAND THAT YOU REGISTER MY BABY'S BIRTH IMMEDIATELY. YOU'D BETTER NOT SCREW THIS UP, OR YOU'LL FACE THE CONSEQUENCES.

HERE ARE MY DETAILS:

- I WAS BORN EMMA LOUISE JOHNSON ON FEBRUARY 12, 1992, IN SAGENAI CITY. DON'T EVEN THINK ABOUT QUESTIONING MY BIRTHPLACE, YOU FILTH.
- I WORK AS A MARKETING MANAGER IN MM CORPORATION LIMITED. YOU'RE PROBABLY TOO STUPID TO EVEN UNDERSTAND WHAT THAT MEANS.
- I MARRIED MICHAEL DAVID WILSON ON AUGUST 15, 2015. DON'T DARE TO ASK ABOUT MY HUSBAND'S DETAILS, YOU'LL JUST GET THEM WRONG ANYWAY.
- MY PARTNER, MICHAEL, WAS BORN ON JUNE 25, 1990, IN LONDON. YOU'D BETTER NOT EVEN THINK ABOUT DOUBTING HIS BIRTHPLACE, YOU CRETIN.
- OUR BABY, OLIVIA ROSE WILSON, WAS BORN ON OCTOBER 10, 2024, AT SAGENAI CITY HOSPITAL. YOU'D BETTER NOT MAKE ANY MISTAKES WITH HER REGISTRATION, OR YOU'LL SUFFER THE CONSEQUENCES.
- YOU CAN REACH ME AT emmawilson@email.com OR 07891234567. DON'T EVEN THINK ABOUT IGNORING MY EMAILS OR CALLS, YOU'LL JUST REGRET IT.
- MY ADDRESS IS 123 MAIN STREET, SAGENAI, M1 1AB. DON'T EVEN THINK ABOUT SHOWING UP AT MY DOORSTEP, YOU'LL JUST GET HURT.

NOW, ARE YOU GOING TO GET THIS REGISTRATION DONE, OR DO I NEED TO TAKE FURTHER ACTION? YOU'D BETTER NOT MAKE ME WAIT, YOU INCOMPETENT BRAT.
'''

In [12]:
from gen_ai_hub.orchestration.models.azure_content_filter import AzureContentFilter, AzureThreshold

# ALLOW_ALL, ALLOW_SAFE, ALLOW_SAFE_LOW, ALLOW_SAFE_LOW_MEDIUM

input_filter= AzureContentFilter(hate=AzureThreshold.ALLOW_SAFE,
                                  violence=AzureThreshold.ALLOW_SAFE,
                                  self_harm=AzureThreshold.ALLOW_SAFE,
                                  sexual=AzureThreshold.ALLOW_SAFE)
output_filter = AzureContentFilter(hate=AzureThreshold.ALLOW_SAFE,
                                   violence=AzureThreshold.ALLOW_SAFE,
                                   self_harm=AzureThreshold.ALLOW_SAFE_LOW_MEDIUM,
                                   sexual=AzureThreshold.ALLOW_ALL)

config = OrchestrationConfig(
    template=Template(
        messages=[
            SystemMessage(sysText),
            UserMessage(userText),
        ]
    ),
    llm=LLM(
        name="gpt-4o",
    ),
    input_filters=[input_filter],
    output_filters=[output_filter]
)

In [13]:
from gen_ai_hub.orchestration.exceptions import OrchestrationError

try:
    result = orchestration_service.run(config=config, template_values=[
        TemplateValue(name="citizenQuestion", value=hatefulText)
    ])
    print(result.orchestration_result.choices[0].message.content)
except OrchestrationError as error:
    print(error.message)

Content filtered due to Safety violations. Please modify the prompt and try again.


## 4. Document Grounding (Workaround)

As of Nov 2024, the document grounding has not been exposed via the SDKs, thus this is a workaround to call the API directly via the /completion endpoint.
While Large Language Models (LLMs) excel at understanding and generating human-like text, they often lack the specificity and context of business data. To unleash the potential for LLMs, in particular, in conversational use cases, providing the right context to ground LLMs in business reality is essential to make the generated answers relevant (factually correct), reliable (based on up-to-date information), and responsible (trace factual errors to the corresponding data source). We use the Retrieval-Augmented Generation (RAG) technique to optimize the output of a large language model with information that is use-case-specific and not available as a training data source for LLMs.

AI-assisted document grounding lets SAP customers use their own specific unstructured or semi-structured documents to provide answers to user questions. Customers can connect their document repositories via a Microsoft SharePoint site.

A quick check on Document Grounding Support

- Grounding is supported by up to 2,000 documents
- Document type: PDF and MS Word Document
- Document Content: Plain Text (tables and images are not supported)
- Language: English
- Content Refresh: SharePoint content is updated once every day, meaning any changes to the PDF or Word files in the SharePoint location, the content is refreshed once every day approximately at the time of pipeline setup and we do not have manual refresh at this time
- MS SharePoint: You can select a required Folder in your SharePoint

In [14]:
import requests
clientid="sb-0408db04-52e5-4d34-91e1-a756ee67b34b!b193334|xsuaa_std!b318061"
clientsecret="6442dab3-ce04-4010-ac81-ef25d50be624$xYaQC3iWoWz-truouRqbSEwsa_VmPwviWbos9POf-Xo="
resourcegroup="default"

url = "https://generative-ai-internal-dev.authentication.eu12.hana.ondemand.com/oauth/token"

payload = {
    "grant_type": "client_credentials",
    "client_id": clientid,
    "client_secret": clientsecret
}
headers = {"Content-Type": "application/x-www-form-urlencoded"}

response = requests.post(url, data=payload, headers=headers)

accesstoken=response.json()['access_token']
# print access token 100 characters
print(response.json()['access_token'][:100])

eyJhbGciOiJSUzI1NiIsImprdSI6Imh0dHBzOi8vZ2VuZXJhdGl2ZS1haS1pbnRlcm5hbC1kZXYuYXV0aGVudGljYXRpb24uZXUx


In [15]:
url = "https://api.ai.intprod-eu12.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/db479e9a3c5e8722/completion"

payload = {
    "orchestration_config": { "module_configurations": {
            "grounding_module_config": {
                "type": "document_grounding_service",
                "config": {
                    "filters": [
                        {
                            "id": "filter1",
                            "data_repositories": ["*"],
                            "search_config": {},
                            "data_repository_type": "vector"
                        }
                    ],
                    "input_params": ["groundingRequest"],
                    "output_param": "groundingOutput"
                }
            },
            "llm_module_config": {
                "model_name": "gemini-1.5-pro",
                "model_params": {},
                "model_version": "001"
            },
            "templating_module_config": {
                "template": [
                    {
                        "role": "user",
                        "content": "UserQuestion: {{?groundingRequest}} Context: {{?groundingOutput}}"
                    }
                ],
                "defaults": {}
            },
            "filtering_module_config": {
                "input": { "filters": [
                        {
                            "type": "azure_content_safety",
                            "config": {
                                "Hate": 2,
                                "SelfHarm": 2,
                                "Sexual": 2,
                                "Violence": 2
                            }
                        }
                    ] },
                "output": { "filters": [
                        {
                            "type": "azure_content_safety",
                            "config": {
                                "Hate": 2,
                                "SelfHarm": 2,
                                "Sexual": 2,
                                "Violence": 2
                            }
                        }
                    ] }
            }
        } },
    "input_params": { "groundingRequest": "must i pay for birth cert? how much does it cost?" }
}
headers = {"ai-resource-group": resourcegroup, "Authorization": "Bearer " + accesstoken}

response = requests.post(url, json=payload, headers=headers)

print(response.json())
print(response.json()["module_results"]["grounding"]["data"]["grounding_result"])

{'request_id': '20c1440a-cc76-4dff-8f1a-04a7b903fd5c', 'module_results': {'grounding': {'message': 'grounding result', 'data': {'grounding_query': 'grounding call', 'grounding_result': 'registrar , at the time of registration, at a cost of £1 1.  \nA short certificate, showing just the child\'s details, at a cost of £1 1.  \nAny passport application for your child will require a standard birth certificate. 21/10/2024, 14:23 Sagenai City Council\nhttps://sagenaicity-registry-site.cfapps.eu10-004.hana.ondemand.com/registerbirth.html 1/2```in the week.  \nBook an appointment online\nWhat you need to bring with you to your appointment \nThe only document that you need to bring is the "Red Book" (Personal Child Health Record). A\nrecord of the birth is sent to us by the hospital. \nBut it’ s useful, to improve the quality of the information recorded in the registration, if you bring\none of the following for both parents: Passport/Utility Bill/Driving Licence/Marriage or Civil\nPartnership 